In [519]:
import locale
import sys
import codecs
import gensim
import os
import random
import csv
import sklearn
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple
from nltk import word_tokenize

In [520]:
os.chdir('H:/GEMINI/Results/WATSON/')
f = open('nlp.data.csv', newline = "")
full = csv.reader(f)
full = list(full)
full = full[1:]

alldocs = []
for line_no, line in enumerate(full):
    words = gensim.utils.to_unicode(line[1].lower()).split()
    tags = [line_no]
    contrast = [line[2]]
    alldocs.append(gensim.models.doc2vec.TaggedDocument(words, tags))

#train_docs = [doc for doc in alldocs if doc.split == 'train']
#test_docs = [doc for doc in alldocs if doc.split == 'test']
#doc_list = alldocs[:]  # for reshuffling per pass

#print('%d docs: %d train-sentiment, %d test-sentiment' % (len(doc_list), len(train_docs), len(test_docs)))

In [521]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

In [527]:
model_dm = gensim.models.Doc2Vec(alpha = 0.1, size = 20, min_alpha = 0.025, iter= 10, min_count = 3, window = 10)
model_dbow = gensim.models.Doc2Vec(alpha = 0.1, size = 20, min_alpha = 0.025, iter= 10, min_count = 13, window = 10, dm= 0)
model_dm.build_vocab(alldocs)
model_dbow.build_vocab(alldocs)

In [528]:
random.shuffle(alldocs)
model_dm.train(alldocs)
model_dbow.train(alldocs)

#for epoch in range(len(alldocs)):
#    if epoch % 20 == 0:
#        print ('Now training epoch %s'%epoch)
#    model.train
    #model.alpha -= 0.002  # decrease the learning rate
    #model.min_alpha = model.alpha  

ValueError: You must specify either total_examples or total_words, for proper alpha and progress calculations. The usual value is total_examples=model.corpus_count.

In [526]:
alldocs[0]

TaggedDocument(words=['chest', 'ct', 'enhanced', '2.5', 'mm', 'axial', 'images', 'every', '2', 'mm', 'were', 'obtained', '(', 'with', 'sagittal', 'and', 'coronal', 'reformats', ')', 'with', '50%', 'overlap', 'and', 'compared', 'to', 'the', 'study', 'obtained', 'in', 'september,', '2013.', 'there', 'is', 'no', 'significant', 'change', 'in', 'the', 'moderate', 'to', 'large', 'right', 'pleural', 'effusion.', 'there', 'is,', 'however,', 'a', 'new', 'smaller', 'left', 'effusion.', 'there', 'are', 'multiple', 'foci', 'of', 'groundglass/interstitial', 'thickening,', 'mainly', 'peripheral', 'and', 'most', 'pronounced', 'in', 'the', 'left', 'upper', 'lobe', '(', 'e.g.', 'image', '72', 'and', 'coronal', 'image', '78', ').', 'these', 'are', 'similar', 'to', 'the', 'september', 'study', 'but', 'more', 'pronounced.', 'similar', 'findings', 'at', 'the', 'left', 'base', 'adjacent', 'to', 'the', 'small', 'effusion', '(', 'e.g.', 'image', '95', ')', 'look', 'similar', 'to', 'the', 'previous', 'study.',

In [504]:
model = model_dbow

In [469]:
model = model_dm

In [505]:
all_data = []
word_vec = []
label = []
for i in range(len(full)):
    word_vec.append(model.infer_vector(full[i][1]))
    label.append(full[i][2])


In [506]:
import numpy as np
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(C=1e5, penalty = "l1", n_jobs = -1)

In [507]:
logisticmodel = log_model.fit(X = word_vec[:200], y = label[:200])

In [508]:
log_model.score(word_vec[200:], label[200:])

0.62244897959183676

In [500]:
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [501]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]
classifiers = [
    KNeighborsClassifier(20),
    SVC(kernel="linear", C=1),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
    DecisionTreeClassifier(max_depth=10),
    RandomForestClassifier(max_depth=10, n_estimators=10, max_features=3),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
X_train, X_test, y_train, y_test = word_vec[:220], word_vec[220:], label[:220], label[220:]
for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(name,  score)


Nearest Neighbors 0.602564102564
Linear SVM 0.628205128205
RBF SVM 0.628205128205
Gaussian Process 0.628205128205
Decision Tree 0.564102564103
Random Forest 0.5
Neural Net 0.628205128205
AdaBoost 0.538461538462
Naive Bayes 0.564102564103
QDA 0.589743589744
